In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
  temperature=0.1,
  streaming=True,
  callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
  ("system","You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients"),
  ("human","I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

In [2]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
  ("system","You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. you don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
  ("human","{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
  "cuisine":"indian",
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon ginger, minced
- 1 can (14 oz) tomato sauce
- 1 tablespoon garam masala
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon paprika
- 1/2 teaspoon turmeric
- 1/2 teaspoon cayenne pepper (adjust to taste)
- Salt and pepper to taste
- Fresh cilantro, chopped for garnish

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can easily substitute the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare these alternatives:\n\n1. **Tofu**: \n   - Use firm or extra-firm tofu for this recipe. Press the tofu to remove excess water by wrapping it in a clean kitchen towel and placing a heavy object on top for about 15-20 minutes.\n   - Cut the tofu into bite-sized cubes and follow the same marinating process as you would with chicken. Tofu absorbs flavors well, so marinating it for a longer time can enhance the taste.\n   - Instead of baking, you can pan-fry the marinated tofu in a bit of oil until it's golden brown and slightly crispy.\n\n2. **Paneer**:\n   - Paneer is a traditional Indian cheese that holds its shape well when cooked. You can find it in Indian grocery stores or make it at home by curdling hot milk with lemon juice or vinegar.\n   - Cut the paneer into cubes and lightly pan-fry them 